In [33]:
# LIBRARIES
# TEXT PROCESSING
import nltk
from nltk.corpus import stopwords
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# DATA SCIENCE
import pandas as pd
import numpy as np

# VISUALIZATION
from wordcloud import WordCloud
from matplotlib import pyplot as plt
get_ipython().magic(u'matplotlib inline')
plt.style.use("ggplot")

# ELSE
from collections import Counter
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [34]:
# FUNCTION TO CLEAN UP BODY TEXT
def cleanUpText(text, additional_stopwords=[]):
    # REMOVE MARK UP
    new_text = text.replace("\r", "").replace("\n", "")
    
    # REMOVE URLS
    new_text = re.sub(r"\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*", "", new_text).strip()
    
    # REMOVE PUNCTUATION
    new_text = new_text.translate(None, string.punctuation)
    
    # REMOVE NUMBERS
    new_text = re.sub(r"\d+", "", new_text)
    
    # LOWERCASE
    new_text = new_text.lower()
    
    #SPLIT
    new_text = new_text.split()
    
    # REMOVE STOPWORDS
    stops = stopwords.words("english") + additional_stopwords
    
    return [word for word in new_text if word not in stops]



# FUNCTION TO CLEAN UP BODY TEXT
def cleanUpTitle(text):
    # REMOVE MARK UP
    new_text = text.replace("\r", "").replace("\n", "")
    
    # REMOVE NUMBERS
    new_text = re.sub(r"\d+", "", new_text)
    
    # LOWERCASE
    new_text = new_text.lower()
    
    return(new_text)

In [42]:
petnlp.shape

(264, 6)

In [35]:
# READ IN DATA TO TRAIN MODEL
petnlp = pd.DataFrame.from_csv("petnlp.csv")

cleaned_titles = [cleanUpTitle(x) for x in petnlp["title"]]


# DEFINE NEW STOP WORDS
new_stops = "President, president, people, without, needs, since, used, get, would, us, united, states, people, american, americans, national, government, petition, make, also, many, must, need, change, ask, use, every, trump, white, house, america, America, executive, Executive"
new_stops = new_stops.split(", ")


tokens = [cleanUpText(x, new_stops) for x in petnlp["body"]]

blobs = [unicode(" ".join(x), errors="replace") for x in tokens]
blobs = [x.encode("ascii", "replace") for x in blobs]
blobs_df = pd.DataFrame({"title":cleaned_titles, "blobs":blobs, "ideology":petnlp["ideology"]})


# REMOVE THE NULLS
index = blobs_df["ideology"].index[blobs_df["ideology"].apply(pd.isnull)]
blobs_df = blobs_df[~pd.isnull(blobs_df["ideology"])]
blobs_df = blobs_df.reset_index(drop=True)

# CHOOSE TRAINING DATA
train = [32, 262, 136, 240, 22, 24, 33, 66, 115, 84, 197, 246, 127, 88, 125, 224, 90, 23, 211, 168, 249, 156, 164, 239, 186, 155, 29, 185, 234, 232, 192, 218, 74, 113, 14, 172, 161, 47, 85, 244, 56, 207, 105, 46, 44, 142, 13, 199, 117, 7, 67, 18, 221, 145, 189, 71, 179, 242, 171, 52, 60, 152, 195, 162, 86, 147, 120, 245, 139, 256, 93, 220, 8, 62, 91, 82, 75, 96, 94, 223, 39, 151, 5, 250, 188, 3, 34, 35, 16, 259, 203, 124, 77, 238, 11, 101, 37, 194, 233, 76, 243, 149, 102, 236, 258, 260, 166, 158, 116, 144, 108, 205, 248, 57, 21, 235, 80, 255, 61, 150, 252, 38, 253, 17, 89, 148, 68, 123, 12, 58, 109, 170, 231, 191, 196, 119, 209, 237, 6, 122, 78, 103, 206, 28, 217, 20, 48, 54, 204, 193, 69, 43, 143, 112, 118, 1, 154, 219, 30, 190, 169, 0, 176, 198, 180, 19, 59, 137, 107, 135, 175, 131, 36, 27, 153, 140, 216, 2, 261, 126, 104, 110, 51, 87, 187, 121, 98, 132, 95, 79, 247, 50, 92, 133, 70, 157, 167, 214, 159, 163, 184, 160, 15, 97, 178, 225, 202, 173, 26, 63]

# VECTORIZING TRAINING DATA
vectorizer = CountVectorizer(analyzer= "word",
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 5000)
train_data_features = vectorizer.fit_transform(blobs_df["blobs"][train])
train_data_features = train_data_features.toarray()


# GENERATE OUR FOREST
forest = RandomForestClassifier(n_estimators = 100)
our_forest = forest.fit(train_data_features, blobs_df["ideology"][train])

/Users/EDIE/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [38]:
# CHECK IF REMOVAL WORKED
blobs_df["ideology"][index]

41    Conservative
Name: ideology, dtype: object

In [39]:
# LOAD IN SOME TEST DOCUMENTS
trump = '''
We will not fail. Our country will thrive and prosper again.
We stand at the birth of a new millennium, ready to unlock the mysteries of space, to free the Earth from the miseries of disease, and to harness the energies, industries and technologies of tomorrow.
A new national pride will stir our souls, lift our sights, and heal our divisions.
It is time to remember that old wisdom our soldiers will never forget: that whether we are black or brown or white, we all bleed the same red blood of patriots, we all enjoy the same glorious freedoms, and we all salute the same great American Flag.
And whether a child is born in the urban sprawl of Detroit or the windswept plains of Nebraska, they look up at the same night sky, they fill their heart with the same dreams, and they are infused with the breath of life by the same almighty Creator.
So to all Americans, in every city near and far, small and large, from mountain to mountain, and from ocean to ocean, hear these words:
You will never be ignored again.
Your voice, your hopes, and your dreams will define our American destiny. And your courage and goodness and love will forever guide us along the way.
Together, We will make America strong again.
We will make wealthy again.
We will make America proud again.
We will make America safe again.
And yes, together, we will make America great again. Thank you. God bless you. And God bless America.'''

hitler = '''
Democratic ideals are a big topic of discussion right now; not in Germany, but other parts of the world talk about them. We in Germany have learned our lesson with democratic ideals; if the rest of the world praises these ideals, we can only answer that the German people had the chance to live within the purest form of this ideal, and we ourselves are now reaping the legacy left by this democracy. We then get a lecture on the wonderful goals of war, especially from the British side. Great Britain has much experience in proclaiming goals of war, considering they have waged more wars than anyone else. The goals they proclaim today are fantastic: the creation of a new Europe. This Europe will be a just place, and the general equality will make arms unnecessary, so we can all disarm. This disarmament is supposed to kick start a period of economic blossoming, trade and movement should commence, especially trade, much trade, free trade! And from this trade, culture is supposed to bloom, and not just culture, but religion, too. In one phrase: the golden age is supposed to dawn. Unfortunately, this golden age has been described in a very similar fashion on several occasions, and not even by prior generations, but by the same people that are describing it yet again today. It's like a very worn-out groove on an old LP. We should pity these gentlemen, who haven't found a new, big idea to hook the people on, because they promised the same things in 1918: the goal of war then was also a "new Europe" and a "new equality", this new equality whose main element is abolishing a nation's right to self-determination. At that time, an equality that would make arms unnecessary in the future was promised. Thence issued the program of disarmament of everyone. And to make this disarmament especially manifest, it was supposed to be crowned by a union of all disarmed states, which had decided that, in the future, all differences (at least no one doubted there would still be differences) between them should be, well, as it is the custom among democracies, be talked to pieces in open discussions. Under no circumstances should there be any more shooting. And at that time it was already said that the consequences of this disarmament and this worldwide parliament would be an incredible blossoming, a blooming of industry and especially (and much emphasis is always put on this) of trade, of free trade. Culture, as well, should not be disregarded in this process, and while one spoke a little bit less about religion at the end of the war than at the beginning, we at least were told, in the year 1918, that it would be a blessed era that God would smile upon.
'''

matloff = '''
Let me offer three obvious (to me, at least) facts:

No one, even the most extreme Republicans, wants to see people dying in the streets. The federal government does have a role, indeed a responsibility. It’s too big for the states, and even if the feds give them block grants, some states, say in the South, would adopt draconian policies.
Many people, especially the stereotypical Trump supporters, live paycheck to paycheck. Tax breaks, health savings accounts and the like are useless to this large group of people. It’s an insult to offer them such things, a let-them-eat-cake solution.
Obamacare was a bold step that filled a void that desperately needed to be filled. But it was built on various assumptions that apparently were shaky. It is not sustainable in its current form. Already some insurers have withdrawn from certain markets.
The Republicans now are making their own shaky assumptions, e.g. that a 30% penalty, for those who forego insurance but then suddenly need it, is the right “price” for not doing their part to support the system. Really? Why shouldn’t it be 5% or 50%, say? No one knows.
'''


nyt = '''
But a senior White House official said that Donald F. McGahn II, the president’s chief counsel, was working to secure access to what Mr. McGahn believed to be an order issued by the Foreign Intelligence Surveillance Court authorizing some form of surveillance related to Mr. Trump and his associates.

The official offered no evidence to support the notion that such an order exists. It would be a highly unusual breach of the Justice Department’s traditional independence on law enforcement matters for the White House to order it to turn over such an investigative document.

Any request for information from a top White House official about a continuing investigation would be a stunning departure from protocols intended to insulate the F.B.I. from political pressure. It would be even more surprising for the White House to seek information about a case directly involving the president or his advisers, as does the case involving the Russia contacts.
'''

robert_reich = '''
Trump and his White House don’t argue on the merits. They attack the credibility of the institutions that come up with facts and arguments they don’t like. 

They even do it preemptively. Last week, White House press secretary Sean Spicer warned that the nonpartisan Congressional Budget Office couldn’t be trusted to come up with accurate numbers about the costs and coverage of the Republican’s replacement for the Affordable Care Act.

“If you’re looking at the CBO for accuracy, you’re looking in the wrong place,” he said.

So what’s the right place? The Oval Office? 

Bear in mind the director of the CBO is a Republican economist and former George W. Bush administration official who was chosen for his position by the Republican Congress in 2015. 

No matter. The White House is worried about what the CBO will say about Trumpcare, so it throws the CBO under the bus before the bus arrives. 

Trump couldn’t care less about the long-term consequences, but the rest of us should. For more than four decades the U.S. budget process has depended on the CBO’s analyses and forecasts. The office has gained a reputation for honesty and reliability under both Republican and Democratic appointees. Now, it’s tainted. 
'''
nate_silver = '''
This is not to say the election was a toss-up in mid-October, which was one of the high-water marks of the campaign for Clinton. But while a Trump win was unlikely, it should hardly have been unthinkable.1 And yet the Times, famous for its “to be sure” equivocations,2 wasn’t even contemplating the possibility of a Trump victory.3
It’s hard to reread this coverage without recalling Sean Trende’s essay on “unthinkability bias,” which he wrote in the wake of the Brexit vote. Just as was the case in the U.S. presidential election, voting on the referendum had split strongly along class, education and regional lines, with voters outside of London and without advanced degrees being much more likely to vote to leave the EU. The reporters covering the Brexit campaign, on the other hand, were disproportionately well-educated and principally based in London. They tended to read ambiguous signs — anything from polls to the musings of taxi drivers — as portending a Remain win, and many of them never really processed the idea that Britain could vote to leave the EU until it actually happened.

So did journalists in Washington and London make the apocryphal Pauline Kael mistake, refusing to believe that Trump or Brexit could win because nobody they knew was voting for them? That’s not quite what Trende was arguing. Instead, it’s that political experts4 aren’t a very diverse group and tend to place a lot of faith in the opinions of other experts and other members of the political establishment. Once a consensus view is established, it tends to reinforce itself until and unless there’s very compelling evidence for the contrary position. Social media, especially Twitter, can amplify the groupthink further. It can be an echo chamber.

I recently reread James Surowiecki’s book “The Wisdom of Crowds” which, despite its name, spends as much time contemplating the shortcomings of such wisdom as it does celebrating its successes. Surowiecki argues5 that crowds usually make good predictions when they satisfy these four conditions:

Diversity of opinion. “Each person should have private information, even if it’s just an eccentric interpretation of the known facts.”
Independence. “People’s opinions are not determined by the opinions of those around them.”
Decentralization. “People are able to specialize and draw on local knowledge.”
Aggregation. “Some mechanism exists for turning private judgments into a collective decision.”
Political journalism scores highly on the fourth condition, aggregation. While Surowiecki usually has something like a financial or betting market in mind when he refers to “aggregation,” the broader idea is that there’s some way for individuals to exchange their opinions instead of keeping them to themselves. And my gosh, do political journalists have a lot of ways to share their opinions with one another, whether through their columns, at major events such as the political conventions or, especially, through Twitter.

But those other three conditions? Political journalism fails miserably along those dimensions.
'''

kuhn = '''
👏🏽👏🏽👏🏽HEY👏🏽👏🏽👏🏽
i hate pleading but please, we have got to stop paying attention to the federal and states' governments' latest outrage--they're going to keep coming, that's the point, to keep us stunned and unable to react--and pay attention to what we can do for the people in our neighborhood because it is getting very bad, very fast. we must start making connections with our neighbors and coworkers, pooling and allocating shared resources among our community to protect the vulnerable, and start forcing accountability from bottom-level local government, because *those* are the things that are going to determine what kind of year 2018 is.

living past 33 is probably the wrong choice overall

one thing you could do if you are concerned about the imminent death of the US poverty relief effort is to call or get in contact with those offices in your area and volunteer while you can, and then continue voluteering after the programs have been exsanguinated; take up collections in your community to fund them, and enlist active help when possible. also, get in contact with local churches, as they'll be the only organized effort at mitigating starvation and homelesness in short order.

It's time to admit that our government has entirely abandoned us. We have to care for each other now, if we care at all, which means first and foremost sharing what we have with those who don't have anything. Pursue this goal actively. Talk to your neighbors, especially those that don't live under rooves.
'''

abe_lincoln = '''Fourscore and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate-we can not consecrate-we can not hallow-this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us-that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion-that we here highly resolve that these dead shall not have died in vain-that this nation, under God, shall have a new birth of freedom-and that government of the people, by the people, for the people shall not perish from the earth.'''

In [40]:
# HOW TO CLEAN TEXT
new_text = [trump, hitler, matloff, nyt, robert_reich, nate_silver, abe_lincoln, kuhn]
new_text_ppl = ["trump", "hitler", "matloff", "michaels_nyt", "reich", "silver", "lincoln", "kuhn"]

new_text_tokens = [cleanUpText(str(x), new_stops) for x in new_text]
new_text_blobs = [" ".join(x) for x in new_text_tokens]


new_text__data_features = vectorizer.transform(new_text_blobs)
new_text__data_features = new_text__data_features.toarray()
new_text__data_features



# PREDICT NEW TEXT DATA
new_text_result = forest.predict(new_text__data_features)

# PREDICTED NEW TEXT DATA PROBABILITIES
new_text_predicted_probs = forest.predict_proba(new_text__data_features)
results_df = pd.DataFrame(new_text_predicted_probs, new_text_ppl)
results_df.columns = ["conserative", "liberal", "neutral"]
results_df

/Users/EDIE/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


,conserative,liberal,neutral
trump,0.42,0.33,0.25
hitler,0.27,0.33,0.40
matloff,0.35,0.49,0.16
michaels_nyt,0.21,0.58,0.21
reich,0.17,0.53,0.30
silver,0.33,0.44,0.23
lincoln,0.40,0.40,0.20
kuhn,0.35,0.42,0.23
